# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [109]:
problem = '''A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. '''

## 2. Ask for parameters

In [110]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [111]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

* **xij:** Liters of component *i* used in the production of product *j*, where:
    * *i* = 1, 2 (C1, C2)
    * *j* = 1, 2, 3 (GCA, GCB, GCC)

This gives us the following decision variables:

* **x11:** Liters of C1 used in GCA production
* **x12:** Liters of C1 used in GCB production
* **x13:** Liters of C1 used in GCC production
* **x21:** Liters of C2 used in GCA production
* **x22:** Liters of C2 used in GCB production
* **x23:** Liters of C2 used in GCC production 


# 2. Ask for objective

In [112]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [113]:
Markdown(response2.text)

Maximize: 

120(x11 + x21) + 135(x12 + x22) + 155(x13 + x23) 


# 3. Ask for constraints

In [114]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [115]:
Markdown(response3.text)

## Constraints:

**Resource Constraints:**

* **C1 Availability:** x11 + x12 + x13 <= 10000 
* **C2 Availability:** x21 + x22 + x23 <= 15000

**Demand Constraints:**

* **GCA Demand:** x11 + x21 >= 6000
* **GCB Demand:** x12 + x22 >= 7000
* **GCC Demand:** x13 + x23 >= 9000

**Critical Element Constraints:**

* **GCA Minimum Critical Element:** 0.4x11 + 0.2x21 >= 0.3(x11 + x21)
* **GCB Maximum Critical Element:** 0.4x12 + 0.2x22 <= 0.3(x12 + x22)

**Component Ratio Constraint:**

* **GCC Minimum C1/C2 Ratio:** x13 >= 0.3x23 

**Non-Negativity Constraints:**

* **All variables:** x11, x12, x13, x21, x22, x23 >= 0 


## 3. Generate the pyomo code

In [116]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [117]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define sets for components and products
components = ['C1', 'C2']
products = ['GCA', 'GCB', 'GCC']

# Define decision variables
model.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Define the objective function
def profit(model):
    return 120*(model.x['C1', 'GCA'] + model.x['C2', 'GCA']) + \
           135*(model.x['C1', 'GCB'] + model.x['C2', 'GCB']) + \
           155*(model.x['C1', 'GCC'] + model.x['C2', 'GCC']) 
model.profit = pyo.Objective(rule=profit, sense=pyo.maximize)

# Define the constraints
model.constraints = pyo.ConstraintList()

# Resource Constraints
model.constraints.add(model.x['C1', 'GCA'] + model.x['C1', 'GCB'] + model.x['C1', 'GCC'] <= 10000) 
model.constraints.add(model.x['C2', 'GCA'] + model.x['C2', 'GCB'] + model.x['C2', 'GCC'] <= 15000)

# Demand Constraints
model.constraints.add(model.x['C1', 'GCA'] + model.x['C2', 'GCA'] >= 6000)
model.constraints.add(model.x['C1', 'GCB'] + model.x['C2', 'GCB'] >= 7000)
model.constraints.add(model.x['C1', 'GCC'] + model.x['C2', 'GCC'] >= 9000)

# Critical Element Constraints
model.constraints.add(0.4*model.x['C1', 'GCA'] + 0.2*model.x['C2', 'GCA'] >= 0.3*(model.x['C1', 'GCA'] + model.x['C2', 'GCA']))
model.constraints.add(0.4*model.x['C1', 'GCB'] + 0.2*model.x['C2', 'GCB'] <= 0.3*(model.x['C1', 'GCB'] + model.x['C2', 'GCB']))

# Component Ratio Constraint
model.constraints.add(model.x['C1', 'GCC'] >= 0.3*model.x['C2', 'GCC'])

# Solve the model
solver = pyo.SolverFactory('glpk') # You can choose a different solver if needed
results = solver.solve(model)

# Display the results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal Solution:")
for i in components:
    for j in products:
        print(f"x[{i},{j}]: {pyo.value(model.x[i,j])}")
print("Profit:", pyo.value(model.profit))
```

No sample data was used in this code. The code defines the mathematical problem as a Pyomo model, including variables, objective function, and constraints directly from the provided problem statement. 


## 4. Input problem data and try running the generated code

In [118]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define sets for components and products
components = ['C1', 'C2']
products = ['GCA', 'GCB', 'GCC']

# Define decision variables
model.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Define the objective function
def profit(model):
    return 120*(model.x['C1', 'GCA'] + model.x['C2', 'GCA']) + \
           135*(model.x['C1', 'GCB'] + model.x['C2', 'GCB']) + \
           155*(model.x['C1', 'GCC'] + model.x['C2', 'GCC']) 
model.profit = pyo.Objective(rule=profit, sense=pyo.maximize)

# Define the constraints
model.constraints = pyo.ConstraintList()

# Resource Constraints
model.constraints.add(model.x['C1', 'GCA'] + model.x['C1', 'GCB'] + model.x['C1', 'GCC'] <= 10000) 
model.constraints.add(model.x['C2', 'GCA'] + model.x['C2', 'GCB'] + model.x['C2', 'GCC'] <= 15000)

# Demand Constraints
model.constraints.add(model.x['C1', 'GCA'] + model.x['C2', 'GCA'] >= 6000)
model.constraints.add(model.x['C1', 'GCB'] + model.x['C2', 'GCB'] >= 7000)
model.constraints.add(model.x['C1', 'GCC'] + model.x['C2', 'GCC'] >= 9000)

# Critical Element Constraints
model.constraints.add(0.4*model.x['C1', 'GCA'] + 0.2*model.x['C2', 'GCA'] >= 0.3*(model.x['C1', 'GCA'] + model.x['C2', 'GCA']))
model.constraints.add(0.4*model.x['C1', 'GCB'] + 0.2*model.x['C2', 'GCB'] <= 0.3*(model.x['C1', 'GCB'] + model.x['C2', 'GCB']))

# Component Ratio Constraint
model.constraints.add(model.x['C1', 'GCC'] >= 0.3*model.x['C2', 'GCC'])

# Solve the model
solver = pyo.SolverFactory('glpk') # You can choose a different solver if needed
results = solver.solve(model)

# Display the results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal Solution:")
for i in components:
    for j in products:
        print(f"x[{i},{j}]: {pyo.value(model.x[i,j])}")
print("Profit:", pyo.value(model.profit))

Status: ok
Termination condition: optimal
Optimal Solution:
x[C1,GCA]: 3730.76923076923
x[C1,GCB]: 3500.0
x[C1,GCC]: 2769.23076923077
x[C2,GCA]: 2269.23076923077
x[C2,GCB]: 3500.0
x[C2,GCC]: 9230.76923076923
Profit: 3525000.0


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [119]:
response.text

'## Variables:\n\n* **xij:** Liters of component *i* used in the production of product *j*, where:\n    * *i* = 1, 2 (C1, C2)\n    * *j* = 1, 2, 3 (GCA, GCB, GCC)\n\nThis gives us the following decision variables:\n\n* **x11:** Liters of C1 used in GCA production\n* **x12:** Liters of C1 used in GCB production\n* **x13:** Liters of C1 used in GCC production\n* **x21:** Liters of C2 used in GCA production\n* **x22:** Liters of C2 used in GCB production\n* **x23:** Liters of C2 used in GCC production \n'

In [120]:
response2.text

'Maximize: \n\n120(x11 + x21) + 135(x12 + x22) + 155(x13 + x23) \n'

In [121]:
response3.text

'## Constraints:\n\n**Resource Constraints:**\n\n* **C1 Availability:** x11 + x12 + x13 <= 10000 \n* **C2 Availability:** x21 + x22 + x23 <= 15000\n\n**Demand Constraints:**\n\n* **GCA Demand:** x11 + x21 >= 6000\n* **GCB Demand:** x12 + x22 >= 7000\n* **GCC Demand:** x13 + x23 >= 9000\n\n**Critical Element Constraints:**\n\n* **GCA Minimum Critical Element:** 0.4x11 + 0.2x21 >= 0.3(x11 + x21)\n* **GCB Maximum Critical Element:** 0.4x12 + 0.2x22 <= 0.3(x12 + x22)\n\n**Component Ratio Constraint:**\n\n* **GCC Minimum C1/C2 Ratio:** x13 >= 0.3x23 \n\n**Non-Negativity Constraints:**\n\n* **All variables:** x11, x12, x13, x21, x22, x23 >= 0 \n'

In [122]:
response4.text

'```python\nimport pyomo.environ as pyo\n\n# Define the model\nmodel = pyo.ConcreteModel()\n\n# Define sets for components and products\ncomponents = [\'C1\', \'C2\']\nproducts = [\'GCA\', \'GCB\', \'GCC\']\n\n# Define decision variables\nmodel.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)\n\n# Define the objective function\ndef profit(model):\n    return 120*(model.x[\'C1\', \'GCA\'] + model.x[\'C2\', \'GCA\']) + \\\n           135*(model.x[\'C1\', \'GCB\'] + model.x[\'C2\', \'GCB\']) + \\\n           155*(model.x[\'C1\', \'GCC\'] + model.x[\'C2\', \'GCC\']) \nmodel.profit = pyo.Objective(rule=profit, sense=pyo.maximize)\n\n# Define the constraints\nmodel.constraints = pyo.ConstraintList()\n\n# Resource Constraints\nmodel.constraints.add(model.x[\'C1\', \'GCA\'] + model.x[\'C1\', \'GCB\'] + model.x[\'C1\', \'GCC\'] <= 10000) \nmodel.constraints.add(model.x[\'C2\', \'GCA\'] + model.x[\'C2\', \'GCB\'] + model.x[\'C2\', \'GCC\'] <= 15000)\n\n# Demand Constraints\nmodel.c